In [8]:
# nbi:hide_in
##########################
# Created on Mar 2020
# @author: juans
##########################

In [9]:
# nbi:hide_in
# Required libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
%matplotlib inline

In [17]:
def dB2mag(x):
    return 10.0**(x / 20.0)

def mag2dB(x):
    return 20.0 * np.log10(np.abs(x))

def dB2pow(x):
    return 10**(x/10.0)

def pow2dB(x):
    return 10 * np.log10(np.abs(x))

In [18]:
# nbi:hide_in
def QuantizeUniform(aNum,nBits):
    #Uniformly quantize signed fraction aNum with nBits
    #Notes:The overload level of the quantizer should be 1.0    
    if(aNum >= 0):
        s = 0
    else:
        s = 1
        
    if(np.abs(aNum) >= 1):
        code = 2**(nBits-1)-1
    else:
        code = int(((2**(nBits)-1)*np.abs(aNum)+1)/2)
        
        
    if(s == 0):
       return int(code)
    else:
       return int(code + 2**(nBits-1))
   
def DequantizeUniform(aQuantizedNum,nBits):
    #Uniformly dequantizes nBits-long number aQuantizedNum 
    #into a signed fraction
    if (aQuantizedNum>>(nBits-1)):
        sign = -1
    else:
        sign = 1
    
    if (sign == -1):
        aNum =  sign*float(2*np.abs(aQuantizedNum - 2**(nBits-1)))/(2**nBits -1)    
    else:
        aNum =  sign*float(2*np.abs(aQuantizedNum))/(2**nBits -1) 

    return aNum

def ScaleFactor(aNum, nScaleBits, nMantBits):
    # Return the floating-point scale factor for a  signed 
    # fraction aNum given nScaleBits scale bits and nMantBits mantissa bits  
    #Notes:
    #The scale factor should be the number of leading zeros
    if nScaleBits < 0: nScaleBits = 0
    if nMantBits <= 0: return 0       # 0 mantissa bits returns 0
    
    maxScale=(1 << nScaleBits) - 1    # max leading 0s scale can id
    maxBits = maxScale + nMantBits    # max bits equiv to this FP rep
    signBit = (1 << (maxBits - 1))    # the location of the sign bit in the Uni Q num
    
    # Uniformly quantize magnitue using maxBits and left shift away sign bit
    code = QuantizeUniform(abs(aNum), maxBits)
    code <<= 1
    # get Scale by shifting left till you hit a 1
    scale = 0
    while scale < maxScale and (signBit & code) == 0:
        code <<= 1
        scale += 1
    
    return scale

def MantissaFP(aNum, scale, nScaleBits, nMantBits):
    # Return the floating-point mantissa for a  signed fraction 
    # aNum given nScaleBits scale bits and nMantBits mantissa bits
    if nMantBits <= 0: return 0
    if nScaleBits < 0: nScaleBits = 0
        
    maxScale = (1 << nScaleBits) - 1
    maxBits = maxScale + nMantBits
    signBit = (1 << (nMantBits - 1))
    
    # Extract sign
    sign = 0
    if aNum < 0:
        sign = 1
        aNum *= -1
        
    # Compute unsigned code using maxBits uniform quantization
    code = QuantizeUniform(aNum, maxBits)
    # extract the mantissa: shift left by scale factor and sign (remove leading 0)
    code <<= (scale + 1)
    # remove leading 1 (if we know it is there) and shfit Left on emore time
    if scale < maxScale:
        code -= (1 << (maxBits - 1)) 
        code <<= 1
    # move bits starting at maxBits down to loest nMantBits - 1
    code >>= (maxBits - nMantBits + 1)
    # add sign to the front of the code
    if sign: code += signBit
        
    return code

def DequantizeFP(scale, mantissa, nScaleBits, nMantBits):
    # Returns a signed fraction for floating point scale and mantissa given
    # Specified scale and mantissa bits
    # zero mantissa bits means zero
    if nMantBits <= 0: return 0
    if nScaleBits < 0: nScaleBits = 0
        
    maxScale = (1 << nScaleBits ) - 1
    maxBits = maxScale + nMantBits
    signBit = (1 << (nMantBits - 1))
    
    if mantissa & signBit:
        sign = 1
        mantissa -= signBit
    else:
        sign = 0
    
    if scale < maxScale:
        mantissa = mantissa + (1 << (nMantBits - 1))
        
    if scale < (maxScale - 1):
        mantissa = (mantissa << 1) + 1
        mantissa <<= (maxScale - scale - 2)
        
    if sign:
        signBit = (1 << (maxBits -1))
        mantissa += signBit
        
    return DequantizeUniform(mantissa, maxBits)

In [30]:
# nbi:hide_in
x = 0.844543

def plot(nScaleBits, nMantBits, level, zoom):
    
    x = dB2mag(level)
    totBits = nScaleBits + nMantBits
    mantMask = 2**(nMantBits) - 1
    scaleMask = (2**(nScaleBits) - 1) << nMantBits;
    

    scale = ScaleFactor(x, nScaleBits, nMantBits)
    mantissa = MantissaFP(x, scale, nScaleBits, nMantBits)
    y = DequantizeFP(scale, mantissa, nScaleBits, nMantBits)
    qStep = ((2**(-nMantBits)) / (2**scale))/2
    
    yAv = np.zeros(2**totBits)
    for i in np.arange(len(yAv)):
        cScale = (i & scaleMask) >> nMantBits
        cMant = i & mantMask
        yAv[i] = DequantizeFP(cScale, cMant, nScaleBits, nMantBits)
            
    zm = zoom
    eb1 = plt.errorbar(0.05, y, yerr=qStep, capsize = 5, color=np.ones(3)*0.5, label='Uncertainty Range')
    eb1[-1][0].set_linestyle(':')
    eb2 = plt.errorbar(-0.05, (y+x)/2, yerr=np.abs(y-x)/2, capsize = 5, color='r', label='Error')
    plt.plot(np.zeros_like(yAv) + 0.05, yAv, '.b', label='Available Steps')
    plt.plot(0, x, 'kx', label='Analog Value')
    plt.plot(0, y, 'xc', label='Quantized Value')
    plt.plot([-1, 1], [0, 0], ':k', lineWidth=2)
    plt.legend()
    plt.ylabel('Amlpliutde')
    plt.xlabel(None)
    plt.xticks([0], ["Some Sample"])
    plt.xlim([-1, 1])
    plt.ylim(np.array([-1, 1]) / zm + x)
    plt.grid(True)

interactive_plot = interactive(plot, nScaleBits=(2, 4, 1), nMantBits=(2, 8, 1), level=(-60, 0, 3), zoom=(1, 100, 10))
interactive_plot

interactive(children=(IntSlider(value=3, description='nScaleBits', max=4, min=2), IntSlider(value=5, descripti…